# İstem Mühendisliğine Giriş
İstem mühendisliği, doğal dil işleme görevleri için istemlerin tasarlanması ve optimize edilmesi sürecidir. Doğru istemleri seçmeyi, bunların parametrelerini ayarlamayı ve performanslarını değerlendirmeyi içerir. İstem mühendisliği, NLP modellerinde yüksek doğruluk ve verimlilik elde etmek için çok önemlidir. Bu bölümde, OpenAI modellerini kullanarak istem mühendisliğinin temellerini inceleyeceğiz.


### Alıştırma 1: Tokenizasyon
OpenAI'nin açık kaynaklı hızlı tokenleştiricisi tiktoken ile Tokenizasyonu keşfedin
Daha fazla örnek için [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) adresine bakabilirsiniz.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Alıştırma 2: Github Models Anahtar Kurulumunu Doğrulama

Aşağıdaki kodu çalıştırarak Github Models uç noktanızın doğru şekilde kurulduğunu doğrulayın. Kod, basit bir istemi dener ve tamamlamayı kontrol eder. `oh say can you see` girdisi, `by the dawn's early light..` şeklinde tamamlanmalıdır.


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Alıştırma 3: Uydurmalar
LLM'den, var olmayan bir konu hakkında ya da önceden eğitildiği veri setinin dışında kalan (daha güncel) konular hakkında bir tamamlayıcı yanıt istediğinizde neler olduğunu keşfedin. Farklı bir istem veya farklı bir model denediğinizde yanıtın nasıl değiştiğine bakın.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Alıştırma 4: Talimat Bazlı
Ana içeriği belirlemek için "text" değişkenini kullanın 
ve bu ana içerikle ilgili bir talimat vermek için "prompt" değişkenini kullanın.

Burada modelden, metni ikinci sınıf bir öğrenciye özetlemesini istiyoruz.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Alıştırma 5: Karmaşık İstek
Sistem, kullanıcı ve asistan mesajlarının olduğu bir istek deneyin
Sistem, asistanın bağlamını belirler
Kullanıcı ve Asistan mesajları, çoklu diyalog bağlamı sağlar

Asistanın kişiliğinin sistem bağlamında "alaycı" olarak ayarlandığına dikkat edin.
Farklı bir kişilik bağlamı kullanmayı deneyin. Ya da farklı bir giriş/çıkış mesajları dizisi deneyin


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Alıştırma: Sezgini Keşfet

Yukarıdaki örnekler, yeni istemler oluşturmak için kullanabileceğin kalıpları gösteriyor (basit, karmaşık, talimat vb.) - örnekler, ipuçları ve daha fazlası gibi konuştuğumuz diğer fikirleri keşfetmek için başka alıştırmalar oluşturmayı dene.



---

**Feragatname**:
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerde hata veya yanlışlıklar bulunabileceğini lütfen unutmayın. Belgenin orijinal diliyle hazırlanmış hali esas alınmalıdır. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından doğabilecek yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
